<a href="https://colab.research.google.com/github/SlowikJakub/Homework4/blob/main/Problem2PartA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [127]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
import collections
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [128]:
from torchvision import datasets, transforms
data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [129]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [130]:
label_map = {6:0, 7:1, 8:2, 9:3}
class_names = ['frog','horse','ship','truck']
cifar2 = [(img, label_map[label])
          for img, label in cifar10 
          if label in [6,7,8,9]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [6,7,8,9]]

In [131]:
import torch.nn as nn
img, _ = cifar2[0]
pool=nn.MaxPool2d(2)
output=pool(img.unsqueeze(0))
img.unsqueeze(0).shape , output.shape

(torch.Size([1, 3, 32, 32]), torch.Size([1, 3, 16, 16]))

In [132]:
model= nn.Sequential(
    nn.Conv2d(3,16, kernel_size=3, padding=1),
    nn.Tanh(),
    nn.MaxPool2d(2),
    nn.Conv2d(3,16, kernel_size=3, padding=1),
    nn.Tanh(),
    nn.MaxPool2d(2),
    nn.Linear(8*8*8,32),
    nn.Tanh(),
    nn.Linear(32,4)
)

In [133]:
import torch.nn as nn
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=nn.Conv2d(3,16, kernel_size=3, padding=1)
    self.act1=nn.Tanh()
    self.pool1=nn.MaxPool2d(2)
    self.conv2=nn.Conv2d(16,8, kernel_size=3, padding=1)
    self.act2=nn.Tanh()
    self.pool2=nn.MaxPool2d(2)
    self.fc1=nn.Linear(8*8*8,32)
    self.act3=nn.Tanh()
    self.fc2=nn.Linear(32,4)
  

  def forward(self,x):
    out = self.pool1(self.act1(self.conv1(x)))
    out = self.pool2(self.act2(self.conv2(out)))
    out = out.view(-1, 8*8*8)
    out = self.act3(self.fc1(out))
    out = self.fc2(out)
    return out

In [134]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [135]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)  # <1>
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

In [136]:
import torch
import torch.nn as nn
import torch.optim as optim
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle =True)

model= Net().to(device=device)
optimizer = optim.SGD(model.parameters(),lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs=200,
    optimizer =optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader=train_loader
)

2021-04-22 23:06:59.092095 Epoch 1, Training loss 1.0699136112444698
2021-04-22 23:07:04.919731 Epoch 10, Training loss 0.4993469786529724
2021-04-22 23:07:11.361872 Epoch 20, Training loss 0.3528493664706477
2021-04-22 23:07:17.734744 Epoch 30, Training loss 0.2798590539172054
2021-04-22 23:07:24.109140 Epoch 40, Training loss 0.23422024113396867
2021-04-22 23:07:30.479777 Epoch 50, Training loss 0.19962730908546203
2021-04-22 23:07:36.854436 Epoch 60, Training loss 0.17615672075948396
2021-04-22 23:07:43.237813 Epoch 70, Training loss 0.15454454340350132
2021-04-22 23:07:49.551665 Epoch 80, Training loss 0.13741389936960924
2021-04-22 23:07:56.000947 Epoch 90, Training loss 0.12206913000002456
2021-04-22 23:08:02.409266 Epoch 100, Training loss 0.10807443782092092
2021-04-22 23:08:08.817503 Epoch 110, Training loss 0.09513300078840682
2021-04-22 23:08:15.226729 Epoch 120, Training loss 0.08345854238342172
2021-04-22 23:08:21.636718 Epoch 130, Training loss 0.0744200095076102
2021-04-

In [137]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64,
                                         shuffle=False)
all_acc_dict = collections.OrderedDict()

def validate(model, train_loader, val_loader):
    accdict = {}
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <1>
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

        print("Accuracy {}: {:.2f}".format(name , correct / total))
        accdict[name] = correct / total
    return accdict

validate(model, train_loader, val_loader)


Accuracy train: 1.00
Accuracy val: 0.91


{'train': 0.99745, 'val': 0.90625}